<a href="https://colab.research.google.com/github/czanalytics/czanalytics/blob/main/logic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logic Programming

We explore ways to use symbolic reasoning, that is  Knowldege Bases, KBs, with facts and rules for logic reasoning.

In addition to basic introduction, we discuss possible use of KB in

- machine learning
- industrial use cases
- business rules

Refs.

- https://github.com/MNoorFawi/pytholog


In [ ]:
!pip install pytholog

In [ ]:
import pytholog as pl

In [ ]:
new_kb = pl.KnowledgeBase("flavor")

new_kb(["likes(noor, sausage)",
        "likes(melissa, pasta)",
        "likes(dmitry, cookie)",
        "likes(nikita, sausage)",
        "likes(assel, limonade)",
        "food_type(gouda, cheese)",
        "food_type(ritz, cracker)",
        "food_type(steak, meat)",
        "food_type(sausage, meat)",
        "food_type(limonade, juice)",
        "food_type(cookie, dessert)",
        "flavor(sweet, dessert)",
        "flavor(savory, meat)",
        "flavor(savory, cheese)",
        "flavor(sweet, juice)",
        "food_flavor(X, Y) :- food_type(X, Z), flavor(Y, Z)",
        "dish_to_like(X, Y) :- likes(X, L), food_type(L, T), flavor(F, T), food_flavor(Y, F), neq(L, Y)"])


In [ ]:
new_kb.query(pl.Expr("likes(noor, sausage)")) # ['Yes']
new_kb.query(pl.Expr("likes(noor, pasta)")) # ['No']

In [ ]:
# query 1
from time import time
start = time()
print(new_kb.query(pl.Expr("food_flavor(What, sweet)")))
print(time() - start)

# [{'What': 'limonade'}, {'What': 'cookie'}]
# 0.0020236968994140625

In [ ]:
# query 2
start = time()
print(new_kb.query(pl.Expr("food_flavor(Food, sweet)")))
print(time() - start)

# [{'Food': 'limonade'}, {'Food': 'cookie'}]
# 0.0

In [ ]:
start = time()
print(new_kb.query(pl.Expr("dish_to_like(noor, What)")))
print(time() - start)

# [{'What': 'gouda'}, {'What': 'steak'}]
# 0.001992940902709961


In [ ]:
start = time()
print(new_kb.query(pl.Expr("dish_to_like(noor, What)")))
print(time() - start)

# [{'What': 'gouda'}, {'What': 'steak'}]
# 0.0

In [ ]:
## new knowledge base object
city_color = pl.KnowledgeBase("city_color")
city_color([
    "different(red, green)",
    "different(red, blue)",
    "different(green, red)",
    "different(green, blue)",
    "different(blue, red)",
    "different(blue, green)",
    "coloring(A, M, G, T, F) :- different(M, T),different(M, A),different(A, T),different(A, M),different(A, G),different(A, F),different(G, F),different(G, T)"
])

In [ ]:
## we will use [0] to return only one answer
## as prolog will give all possible combinations and answers
city_color.query(pl.Expr("coloring(Alabama, Mississippi, Georgia, Tennessee, Florida)"), cut = True)

# {'Alabama': 'blue',
#  'Mississippi': 'red',
#  'Georgia': 'red',
#  'Tennessee': 'green',
#  'Florida': 'green'}

In [ ]:
battery_kb = pl.KnowledgeBase("battery")
battery_kb([
	"battery(dead,P) :- voltmeter(battery_terminals,abnormal,P2), P is P2 + 0.5",
	"battery(dead,P) :- electrical_problem(P), P >= 0.8",
	"battery(dead,P) :- electrical_problem(P2), age(battery,old,P3), P is P2 * P3 * 0.9",
	"electrical_problem(0.7)",
	"age(battery,old, 0.8)",
	"voltmeter(battery_terminals,abnormal,0.3)"])

battery_kb.query(pl.Expr("battery(dead, Probability)"))

# [{'Probability': 0.8}, {'Probability': 'No'}, {'Probability': 0.504}]
# the second one is "No" because the condition has not been met.

In [ ]:
iris_kb = pl.KnowledgeBase("iris")
iris_kb([## Rules
	"species(setosa, Truth) :- petal_width(W), Truth is W <= 0.80",
	"species(versicolor, Truth) :- petal_width(W), petal_length(L), Truth is W > 0.80 and L <= 4.95",
	"species(virginica, Truth) :- petal_width(W), petal_length(L), Truth is W > 0.80 and L > 4.95",
	## New record
	"petal_length(5.1)",
	"petal_width(2.4)"])

In [ ]:
iris_kb.query(pl.Expr("species(Class, Truth)"))

# [{'Class': 'setosa', 'Truth': 'No'},
#  {'Class': 'versicolor', 'Truth': 'No'},
#  {'Class': 'virginica', 'Truth': 'Yes'}]

In [ ]:
iris_kb.query(pl.Expr("species(Class, Truth)"))

# [{'Class': 'setosa', 'Truth': 'No'},
#  {'Class': 'versicolor', 'Truth': 'No'},
#  {'Class': 'virginica', 'Truth': 'Yes'}]

In [ ]:
iris_kb.rule_search(pl.Expr("species(Species, Truth)"))

# [species(setosa,Truth):-petal_width(W),TruthisW<=0.80,
#  species(versicolor,Truth):-petal_width(W),petal_length(L),TruthisW>0.80andL<=4.95,
#  species(virginica,Truth):-petal_width(W),petal_length(L),TruthisW>0.80andL>4.95]

In [ ]:
new_kb.clear_cache()

In [ ]:
import pandas as pd
df = pd.DataFrame({"has_work": ["david", "daniel"], "tasks": [8, 3]})
df
#	has_work  tasks
#0	   david	  8
#1	  daniel	  3
ex = pl.KnowledgeBase()
for i in range(df.shape[0]):
    ex([f"has_work({df.has_work[i]}, {df.tasks[i]})"])

ex.db
# {'has_work': {'facts': [has_work(david,8), has_work(daniel,3)],
#   'goals': [[], []],
#   'terms': [['david', '8'], ['daniel', '3']]}}

In [ ]:
graph = pl.KnowledgeBase("graph")
graph([
	"edge(a, b, 6)", "edge(a, c, 1)", "edge(b, e, 4)",
	"edge(b, f, 3)", "edge(c, d, 3)", "edge(d, e, 8)",
	"edge(e, f, 2)",
	"path(X, Y, W) :- edge(X , Y, W)",
	"path(X, Y, W) :- edge(X, Z, W1), path(Z, Y, W2), W is W1 + W2"])

answer, path = graph.query(pl.Expr("path(a, f, W)"), show_path = True)
print(answer)
print([x for x in path if str(x) > "Z"])

# [{'W': 9}, {'W': 12}, {'W': 14}]
# ['d', 'b', 'e', 'c']

In [ ]:
answer, path = graph.query(pl.Expr("path(a, e, W)"), show_path = True, cut = True)
print(answer)
print([x for x in path if str(x) > "Z"])

# [{'W': 10}]
# ['b']